<a href="https://colab.research.google.com/github/Aryan-2605/DGCS-AI-DecisionSystem/blob/master/DGCS-TargetPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
from shapely.geometry import Point, Polygon
import ast


# **Cell 1: Imports and Data**

In [ ]:
drive.mount('/content/drive')
player_data = pd.read_csv('/content/drive/MyDrive/DGCS/player_data.csv')
hole_data = pd.read_csv('/content/drive/MyDrive/DGCS/Hole_1.csv')
player_simulation = pd.read_csv('/content/drive/MyDrive/DGCS/hole_sim.csv')

display(player_data.head())
display(hole_data.head())
display(player_simulation.head())

Mounted at /content/drive


,player_id,Age,Gender,HCP,Driver,Driver_Dispersion,3-Wood,3-Wood_Dispersion,5-Wood,5-Wood_Dispersion,...,9-Iron,9-Iron_Dispersion,PW,PW_Dispersion,GW,GW_Dispersion,SW,SW_Dispersion,LW,LW_Dispersion
0,1,31,Male,15.0,242.0,26.2,220.9,17.8,NaN,NaN,...,115.9,8.3,114.0,3.9,113.6,6.7,99.7,4.5,NaN,NaN
1,2,39,Male,13.4,232.9,22.9,233.2,22.4,NaN,NaN,...,134.3,6.1,117.8,4.3,124.5,6.5,100.4,4.8,NaN,NaN
2,3,37,Male,29.0,161.6,30.1,127.6,22.5,NaN,NaN,...,100.2,9.2,90.5,8.0,86.0,6.2,71.3,8.5,NaN,NaN
3,4,21,Female,6.4,217.8,20.8,NaN,NaN,184.6,13.7,...,NaN,NaN,108.4,5.3,102.6,5.9,94.8,6.5,NaN,NaN
4,5,42,Male,29.2,150.1,26.0,NaN,NaN,136.9,22.9,...,86.4,10.0,102.2,8.5,NaN,NaN,71.1,7.8,NaN,NaN


,Area,Coordinates
0,Fairway,"[(51.60492078656368, -0.2198113604936469), (51..."
1,TreeLine,"[(51.60430132835705, -0.22023673406949), (51.6..."
2,TreeLine,"[(51.60398427667845, -0.2201013746057723), (51..."
3,TreeLine,"[(51.60436777154526, -0.2199946487339721), (51..."
4,TreeLine,"[(51.60407076234819, -0.2191739058443709), (51..."


,round_id,player_id,hole_id,shot_id,start_coords,end_coords,club,bearing,rating
0,1,1,1,1,"(51.60576426300037, -0.22007174187974488)","(51.6044357422908, -0.2196944790609735)",Driver,286.973786,0.281199
1,1,1,1,2,"(51.6044357422908, -0.2196944790609735)","(51.603432024998135, -0.21899429264472972)",8-Iron,288.025731,0.686447
2,1,1,1,3,"(51.603432024998135, -0.21899429264472972)","(51.60301213993455, -0.21925742493499673)",SW,238.214285,0.998526
3,2,1,1,1,"(51.60576426300037, -0.22007174187974488)","(51.60390037216681, -0.2195181699551096)",Driver,286.973786,0.981531
4,2,1,1,2,"(51.60390037216681, -0.2195181699551096)","(51.60335422443254, -0.21916248136943686)",SW,287.900127,0.814996


# **Cell 2: Preprocessing Datasets**

In [ ]:
import numpy as np
import pandas as pd
from ast import literal_eval
from geopy.distance import geodesic
from shapely.geometry import Point, Polygon
import ast
from shapely.geometry.linestring import LineString

#Area Functions:

class Area():
      def __init__(self, hole_data):
        self.hole_data = hole_data
        self.polygons = self.create_polygon()

      def parse_hole_data(self):
          predefined_areas = ['Fairway', 'TreeLine', 'Green', 'Bunker', 'Zone', 'TeeBox']
          area_coordinates = {}

          for area in predefined_areas:
              arrays = self.hole_data.loc[self.hole_data['Area'] == area, 'Coordinates'].values

              converted = [ast.literal_eval(item) for item in arrays]

              area_coordinates[area] = converted

          return area_coordinates

      def create_polygon(self):
          area_coordinates = self.parse_hole_data()
          hole_polygons = {}

          for zone, coordinates in area_coordinates.items():
              #print(f"Zone: {zone}")
              for i, coords in enumerate(coordinates):
                  polygon = Polygon(coords)
                  hole_polygons.setdefault(zone, []).append(polygon)
                  #print(f"  Polygon {i + 1}: {polygon}")

          return hole_polygons

      def return_location(self, location):
          predefined_areas = ['Fairway', 'TreeLine', 'Green', 'Bunker', 'Zone', 'TeeBox']
          is_inside = {}

          for zone, polygons in self.polygons.items():
              if zone in predefined_areas:
                  is_inside[zone] = False

                  for i, polygon in enumerate(polygons):
                      if isinstance(polygon, str):
                          self.polygons[zone][i] = Polygon(ast.literal_eval(polygon))
                          polygon = self.polygons[zone][i]

                      if polygon.contains(location):
                          is_inside[zone] = True
                          break

          if is_inside.get("TeeBox", True):
            return "TeeBox"
          elif is_inside.get("Bunker", True):
            return "Bunker"
          elif is_inside.get("Green", True):
            return "Green"
          elif is_inside.get("Fairway", True):
            return "Fairway"
          elif is_inside.get("TreeLine", True):
            return "Treeline"
          elif is_inside.get("Zone", True):
            return "Rough"
          else:
            return "Invalid"





Area = Area(hole_data)
Area.create_polygon()

# Merge Table 1 (player_simulation) & Table 2 (player_data) on Player ID
df = player_simulation.merge(player_data, on="player_id", how="left")

df["start_x"], df["start_y"] = zip(*df["start_coords"].apply(lambda x: literal_eval(x) if isinstance(x, str) else x))
df["end_x"], df["end_y"] = zip(*df["end_coords"].apply(lambda x: literal_eval(x) if isinstance(x, str) else x))

# Calculate Derived Features
#df["distance_covered"] = np.sqrt((df["end_x"] - df["start_x"])**2 + (df["end_y"] - df["start_y"])**2)

R = 6371 * 1093.61  # Earth's radius in yards

df["distance_covered"] = 2 * R * np.arcsin(
    np.sqrt(
        np.sin(np.radians(df["end_x"] - df["start_x"]) / 2) ** 2 +
        np.cos(np.radians(df["start_x"])) * np.cos(np.radians(df["end_x"])) *
        np.sin(np.radians(df["end_y"] - df["start_y"]) / 2) ** 2
    )
)

# Ensure deviation_from_target uses the correct dispersion column
#df["deviation_from_target"] = df.apply(lambda row: row[f"{row['club']}_Dispersion"], axis=1)

df["Gender"] = df["Gender"].map({"Male": 0, "Female": 1})  # Convert Gender to binary

#Mapping of the clubs.
club_mapping = dict(enumerate(df["club"].astype("category").cat.categories))


df["club"] = df["club"].astype("category").cat.codes


df["start_zone"] = df.apply(lambda row: Area.return_location(Point(row["start_x"], row["start_y"])), axis=1)
df["end_zone"] = df.apply(lambda row: Area.return_location(Point(row["end_x"], row["end_y"])), axis=1)

#display(df)


all_zones = pd.concat([df["start_zone"], df["end_zone"]]).astype("category").cat.categories
zone_mapping = {zone: i for i, zone in enumerate(all_zones)}

df["start_zone"] = df["start_zone"].map(zone_mapping)
df["end_zone"] = df["end_zone"].map(zone_mapping)


df.drop(columns=[ "start_coords", "end_coords"], inplace=True)
# Maybe Add rating back to the drop table??
# Display preprocessed dataset
display(df)



# Print the club mapping
print("Club Mapping:", club_mapping)

print("Zone Mapping:", zone_mapping)



,round_id,player_id,hole_id,shot_id,club,bearing,rating,Age,Gender,HCP,...,SW_Dispersion,LW,LW_Dispersion,start_x,start_y,end_x,end_y,distance_covered,start_zone,end_zone
0,1,1,1,1,11,286.973786,0.281199,31,0,15.0,...,4.5,NaN,NaN,51.605764,-0.220072,51.604436,-0.219694,164.046672,4,1
1,1,1,1,2,9,288.025731,0.686447,31,0,15.0,...,4.5,NaN,NaN,51.604436,-0.219694,51.603432,-0.218994,133.019885,1,3
2,1,1,1,3,15,238.214285,0.998526,31,0,15.0,...,4.5,NaN,NaN,51.603432,-0.218994,51.603012,-0.219257,54.791103,3,2
3,2,1,1,1,11,286.973786,0.981531,31,0,15.0,...,4.5,NaN,NaN,51.605764,-0.220072,51.603900,-0.219518,230.480159,4,1
4,2,1,1,2,15,287.900127,0.814996,31,0,15.0,...,4.5,NaN,NaN,51.603900,-0.219518,51.603354,-0.219162,71.641295,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32601,8028,4000,1,2,9,291.028682,0.124904,29,0,29.7,...,7.0,NaN,NaN,51.604899,-0.219955,51.604631,-0.219767,35.545997,3,1
32602,8028,4000,1,3,1,288.325027,0.076376,29,0,29.7,...,7.0,NaN,NaN,51.604631,-0.219767,51.604412,-0.219643,28.264033,1,1
32603,8028,4000,1,4,1,286.345975,0.104372,29,0,29.7,...,7.0,NaN,NaN,51.604412,-0.219643,51.604080,-0.219550,40.961366,1,1
32604,8028,4000,1,5,9,286.587228,0.561953,29,0,29.7,...,7.0,NaN,NaN,51.604080,-0.219550,51.603364,-0.219330,88.672418,1,3


Club Mapping: {0: '3-Hybrid', 1: '3-Wood', 2: '4-Hybrid', 3: '4-Iron', 4: '5-Hybrid', 5: '5-Iron', 6: '5-Wood', 7: '6-Iron', 8: '7-Iron', 9: '8-Iron', 10: '9-Iron', 11: 'Driver', 12: 'GW', 13: 'LW', 14: 'PW', 15: 'SW'}
Zone Mapping: {'Bunker': 0, 'Fairway': 1, 'Green': 2, 'Rough': 3, 'TeeBox': 4, 'Treeline': 5}


# **Cell 3: Data Splitting and Target View**

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["club", "hole_id", "bearing", "round_id", "end_x", "end_y", "end_zone", "distance_covered"])
y_lat = df[["end_x"]]
y_lon = df["end_y"]


X_train, X_test, y_lat_train, y_lat_test = train_test_split(X, y_lat, test_size=0.2, random_state=42)
X_train, X_test, y_lon_train, y_lon_test = train_test_split(X, y_lon, test_size=0.2, random_state=42)

display(y_lat)
display(y_lon)


,end_x
0,51.604436
1,51.603432
2,51.603012
3,51.603900
4,51.603354
...,...
32601,51.604631
32602,51.604412
32603,51.604080
32604,51.603364


,end_y
0,-0.219694
1,-0.218994
2,-0.219257
3,-0.219518
4,-0.219162
...,...
32601,-0.219767
32602,-0.219643
32603,-0.219550
32604,-0.219330


# **Cell 5: Train another Model**
## **Model -  XGBoost**

In [ ]:
import time
import numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score



#📌 Best Parameters: {'n_estimators': 500, 'learning_rate': 0.05, 'max_depth': 10, 'subsample': 0.8, 'colsample_bytree': 0.8, 'random_state': 42, 'use_label_encoder': False, 'eval_metric': 'logloss'}


lat_model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100, learning_rate=0.1, max_depth=5)
lat_model.fit(X_train, y_lat_train)

# longitude
lon_model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100, learning_rate=0.1, max_depth=5)
lon_model.fit(X_train, y_lon_train)

y_pred_lat = lat_model.predict(X_test)
y_pred_lon = lon_model.predict(X_test)


#Calc for lat
mae_lat = mean_absolute_error(y_lat_test, y_pred_lat)
mse_lat = mean_squared_error(y_lat_test, y_pred_lat)
r2_lat = r2_score(y_lat_test, y_pred_lat)

#calc for long

mae_lon = mean_absolute_error(y_lon_test, y_pred_lon)
mse_lon = mean_squared_error(y_lon_test, y_pred_lon)
r2_lon = r2_score(y_lon_test, y_pred_lon)


print(f"🏆 Latitude Accuracy 🏆")
print(f"🙈 Mean Absolute Error (MAE): {mae_lat:.4f}")
print(f"🙊 Mean Squared Error (MSE): {mse_lat:.4f}")
print(f"🎯 R² Score: {r2_lat:.4f} \n\n")

print(f'🏆 Longitude Accuracy 🏆' )
print(f"🙈 Mean Absolute Error (MAE): {mae_lon:.4f}")
print(f"🙊 Mean Squared Error (MSE): {mse_lon:.4f}")
print(f"🎯 R² Score: {r2_lon:.4f} \n\n")


🏆 Latitude Accuracy 🏆
🙈 Mean Absolute Error (MAE): 0.0001
🙊 Mean Squared Error (MSE): 0.0000
🎯 R² Score: 0.9744 


🏆 Longitude Accuracy 🏆
🙈 Mean Absolute Error (MAE): 0.0001
🙊 Mean Squared Error (MSE): 0.0000
🎯 R² Score: 0.8390 




# **Cell 7: Analytics on model's features and classification report**

In [ ]:
import pandas as pd

lat_importance = lat_model.feature_importances_
lon_importance = lon_model.feature_importances_

feature_names = X_train.columns

feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Latitude Importance': lat_importance,
    'Longitude Importance': lon_importance
}).sort_values(by="Latitude Importance", ascending=False)

print(feature_importance_df)


                Feature  Latitude Importance  Longitude Importance
38              start_x             0.494423              0.117173
6                Driver             0.090930              0.061812
2                rating             0.089253              0.080406
39              start_y             0.055503              0.495738
5                   HCP             0.052393              0.037978
34                   SW             0.039287              0.034579
28               9-Iron             0.033897              0.015341
30                   PW             0.029263              0.067244
26               8-Iron             0.027681              0.020172
8                3-Wood             0.025624              0.021979
24               7-Iron             0.014005              0.026378
40           start_zone             0.012694              0.000000
22               6-Iron             0.010847              0.021200
16             5-Hybrid             0.005028              0.00

# **Cell 8: Retrive a user and tweak attributes for prediction**

In [ ]:
#Change only these 2 lines for a new user and new location.
start_coords = (51.60451897348522, -0.22000039825044562)
new_user =  player_data.iloc[274:275].copy() #Input is user = user_id:user_id+1


start_zone = Area.return_location(Point(start_coords[0], start_coords[1]))
shot_number = 3

#new_user.drop(columns=['player_id'], inplace = True)
new_user['Gender'] = new_user['Gender'].map({'Male': 0, 'Female': 1})
new_user['rating'] = 0.2
new_user['start_x'] = start_coords[0]
new_user['start_y'] = start_coords[1]

new_user['start_zone'] = zone_mapping[start_zone]
new_user["shot_id"] = shot_number
R = 6371 * 1093.61




new_user

,player_id,Age,Gender,HCP,Driver,Driver_Dispersion,3-Wood,3-Wood_Dispersion,5-Wood,5-Wood_Dispersion,...,GW_Dispersion,SW,SW_Dispersion,LW,LW_Dispersion,rating,start_x,start_y,start_zone,shot_id
274,275,58,0,32.2,149.0,27.9,NaN,NaN,NaN,NaN,...,NaN,60.5,9.0,55.0,9.6,0.2,51.604519,-0.22,5,3


# **Cell 9: Generate Recommendation**

In [ ]:
new_user = new_user[X_train.columns]

predicted_lat = lat_model.predict(new_user)[0]
print(f"📍 Predicted Latitude: {predicted_lat:.6f}")


predicted_lon = lon_model.predict(new_user)[0]
print(f"📍 Predicted Lon: {predicted_lon:.6f}")

print(f'Final Prediction: ({predicted_lat},{predicted_lon})')


📍 Predicted Latitude: 51.604073
📍 Predicted Lon: -0.219776
Final Prediction: (51.60407257080078,-0.21977558732032776)


'\nprint("Random Forest\'s Prediction:")\nprint(f"Recommended Club: {predicted_club} \n")\n\npredicted_club_code = xgb_model.predict(new_user)[0]\npredicted_club = club_mapping[predicted_club_code]\nprint("XRB\'s Prediction:")\nprint(f"Recommended Club: {predicted_club}")\n'